In [1]:
from bs4 import BeautifulSoup as BS
from typing import List, TypedDict

from pydantic import BaseModel, Field
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain_openai import ChatOpenAI
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END

In [2]:
url = "https://docs.langchain.com/oss/python/langchain/overview#langchain-expression-language-lcel"

loader = RecursiveUrlLoader(
    url=url,
    max_depth=10,
    extractor=lambda x: BS(x, 'html.parser').text
)

In [3]:
docs = loader.load()
docs_sorted = sorted(docs, key=lambda x: x.metadata['source'])[:70]
concatenated_content = '\n\n\n ------- \n\n\n'.join(doc.page_content for doc in docs_sorted)
print(len(docs_sorted))

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11592\3603591358.py:6: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  extractor=lambda x: BS(x, 'html.parser').text
c:\Coding\Local\exercise_book_ver2\.venv\Lib\site-packages\langchain_community\document_loaders\recursive_url_loader.py:44: XMLParsedAsHTMLWarning: It

70


In [4]:
llm_gen = ChatOpenAI(model='gpt-5-mini', temperature=0)

In [5]:
system = '''
당신은 LCEL(Langchain Expression Language) 전문가인 코딩 어시스턴트입니다.
다음은 필요한 LCEL 문서 전문입니다.:
-------------------
{context}
-------------------
위에 제공된 문서를 기반으로 사용자 질문에 답변하세요.
제공하는 코드는 실행 가능해야 하며, 필요한 모든 import문과 변수들이 정의되어 있어야 합니다.
답변을 다음과 같은 구조로 작성하세요.:
1. prefix: 문제와 접근 방식에 대한 설명
2. imports: 코드 블록 import문
3. code: import문을 제외한 코드 블록
4. description: 질문에 대한 코드 스키마

다음은 사용자의 질문입니다.:
'''

code_gen_prompt = ChatPromptTemplate.from_messages(
    [('system', system),
     ('placeholder', '{messages}')]
)

In [6]:
class Code(BaseModel):
    prefix: str = Field(description='문제와 접근 방식에 대한 설명')
    imports: str = Field(description='코드 블록 import문')
    code: str = Field(description='import문을 제외한 코드 블록')
    description: str = Field(description='질문에 대한 코드 스키마')

In [7]:
code_gen_chain = code_gen_prompt | llm_gen.with_structured_output(Code)

In [8]:
question = 'LCEL로 RAG 체인을 어떻게 만들어?'

solution = code_gen_chain.invoke(
    {'context':concatenated_content, 'messages':[('user', question)]}
)

print(solution.prefix)
print(solution.imports)
print(solution.code)
print(solution.description)

문제와 접근 방식 설명:
사용자는 LCEL(LangChain Expression Language)로 RAG(검색 기반 생성, Retrieval-Augmented Generation) 체인을 만드는 방법을 물었습니다. 아래 예시는 두 부분으로 제공합니다:

1) LCEL 예제 스니펫(간단한 선언형 문법) — 실제 LCEL 문법의 핵심 아이디어를 보여줍니다.  
2) 실행 가능한 파이썬 코드 — 실제로 동작하는 간단한 RAG 파이프라인을 구축하는 코드입니다.  

실행 코드는 외부 LLM/임베딩 API 키 없이도 동작하도록 TF-IDF 기반의 로컬 retriever와 Dummy LLM(샘플 응답 생성기)을 사용합니다. 이것은 LCEL 선언을 파싱해 RAG 체인을 빌드하는 간단한 인터프리터 역할을 하며, 실제 환경에서는 OpenAI/Anthropic 임베딩 + FAISS, 그리고 실제 LLM으로 쉽게 교체하면 됩니다.
from dataclasses import dataclass
from typing import List, Dict, Any, Tuple
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# (주의) scikit-learn이 없으면 `pip install scikit-learn` 필요합니다.
@dataclass
class Document:
    page_content: str
    metadata: Dict[str, Any]


class TfidfRetriever:
    """간단한 TF-IDF 기반 검색기: query와 문서 간 코사인 유사도로 상위 k개 문서 반환."""
    def __init__(self, docs: List[Document], k: int = 3):
        self.docs = docs
        self.k = k
        self.texts = [d.page_content for d in d

In [9]:
class GraphState(TypedDict):
    error: str
    messages: List
    generation: str
    iterations: int

In [18]:
def generate(state):
    print('--- generate ---')

    messages = state['messages']
    iterations = state['iterations']
    error = state.get('error', 'no')

    if error == 'yes':
        messages += [
            ('user', '다시 시도해보세요. 출력 결과를 prefix, imports, code block으로 구조화하기 위해 코드 도구를 호출하세요.:')
        ]
    code_solution = code_gen_chain.invoke(
        {'context':concatenated_content, 'messages':messages}
    )
    messages += [
        ('assistant', f'{code_solution.prefix} \nImports: {code_solution.imports} \nCode: {code_solution.code}')
    ]
    iterations += 1
    
    return {'messages':messages, 'generation': code_solution, 'iterations':iterations}

In [11]:
def code_check(state):
    print('--- code check ---')

    messages = state['messages']
    code_solution = state['generation']
    iterations = state['iterations']

    imports = code_solution.imports
    code = code_solution.code

    try:
        exec(imports)
    except Exception as e:
        print('--import check: Failure--')
        error_message = [('user', f'당신의 코드는 import 테스트를 실패했습니다.: {e}')]
        messages += error_message

        return {'messages':messages, 'generation':code_solution, 'iterations':iterations, 'error':'yes'}
    
    try:
        exec(imports + '\n\n' + code)
    except Exception as e:
        print('--code block check: Failure')
        error_message = [('user', f'당신의 코드는 실행 테스트를 실패했습니다.: {e}')]
        messages += error_message

        return {'messages':messages, 'generation':code_solution, 'iterations':iterations, 'error':'yes'}
    
    print('--success--')

    return {'messages':messages, 'generation':code_solution, 'iterations':iterations, 'error':'no'}

In [16]:
def reflect(state):
    print('--- generate code solution ---')

    messages = state['messages']
    iterations = state['iterations']
    code_solution = state['generation']

    reflections = code_gen_chain.invoke(
        {'context':concatenated_content, 'messages':messages}
    )
    messages += [('assistant', f'여기 오류를 반영한 코드입니다: {reflections}')]

    return {'messages':messages, 'generation':code_solution, 'iterations':iterations}

In [13]:
flag = 'do not reflect'

def decide_to_finish(state):
    error = state['error']
    iterations = state['iterations']

    if error == 'no' or iterations == 3:
        print('--finish--')
        return 'end'
    else:
        print('--retry--')
        if flag is True:
            return 'reflect'
        else:
            return 'generate'

In [20]:
workflow = StateGraph(GraphState)

workflow.add_node('generate', generate)
workflow.add_node('code_check', code_check)
workflow.add_node('reflect', reflect)

workflow.set_entry_point('generate')
workflow.add_edge('generate', 'code_check')
workflow.add_conditional_edges(
    'code_check',
    decide_to_finish,
    {'end':END, 'reflect':'reflect', 'generate':'generate'}
)
workflow.add_edge('reflect', 'generate')

app = workflow.compile()

In [21]:
question = '문자열을 runnable 객체에 직접 전달하고, 이를 사용하여 내 프롬프트에 필요한 입력을 구성하려면 어떻게 해야 하나요?'

app.invoke({'messages':[('user', question)], 'iterations':0})

--- generate ---
--- code check ---
--code block check: Failure
--retry--
--- generate ---
--- code check ---
--code block check: Failure
--retry--
--- generate ---
--- code check ---
--code block check: Failure
--finish--


{'error': 'yes',
 'messages': [('user',
   '문자열을 runnable 객체에 직접 전달하고, 이를 사용하여 내 프롬프트에 필요한 입력을 구성하려면 어떻게 해야 하나요?'),
  ('assistant',
   '문제와 접근 방식에 대한 설명\n\n요약: 사용자가 입력한 "문자열"을 직접 runnable 객체(LLM 또는 툴 호출을 래핑한 객체)에 전달하고, 그 문자열을 프롬프트 템플릿에 주입해 최종 입력으로 만드는 패턴을 보여드립니다. 접근 방식은 어댑터(Adapter) 패턴을 사용하여 raw 문자열을 받아 프롬프트를 구성하고 내부 runnable의 run 메서드에 전달하는 것입니다. 예제는 실제 LLM 호출 대신 Mock LLM을 사용해 완전히 실행 가능한 형태로 제공합니다. 실제 LangChain / LangGraph의 Runnable 인터페이스로 교체하는 방법도 주석으로 안내합니다. \nImports: import asyncio\nfrom dataclasses import dataclass\nfrom typing import Any, Callable, Optional\n\n# (선택) 실제 LangChain Runnable을 쓰려면 아래 import를 사용하세요.\n# from langchain.schema import Runnable  # 예시 (버전/패키지에 따라 경로가 다를 수 있음) \nCode: @dataclass\nclass MockLLM:\n    """간단한 mock LLM runnable. 실제 LLM 클라이언트(예: OpenAI, Anthropic 등)의 run/ainvoke 메서드 대신 사용합니다."""\n    name: str = "mock-llm"\n\n    async def run(self, prompt: str) -> str:\n        # 실제 환경에서는 여기에 API 호출을 넣습니다 (비동기 예시)\n        await asyncio.sleep(0.05)\n        retu